# Exploring Ebay Car Sales Data
In this project, we will analyse a dataset of used cars from *eBay Kleinanzeigen* in Germany. We'll use a stripped down version of [this dataset from Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data) from the year 2016.

The data dictionary for the provided dataset is as followed:
- `dateCrawled` : when this ad was first crawled, all field-values are taken from this date
- `name` : "name" of the car
- `seller` : private or dealer
- `offerType` : the type of listing
- `price` : the price on the ad to sell the car
- `abtest` : whether the listing is included in an A/B test.
- `vehicleType`
- `yearOfRegistration` : at which year the car was first registered
- `gearbox` : the transmission type.
- `powerPS` : power of the car in PS
- `model` : the car model name
- `kilometer` : how many kilometers the car has driven
- `monthOfRegistration` : at which month the car was first registered
- `fuelType`
- `brand` : the brand of the car
- `notRepairedDamage` : if the car has a damage which is not repaired yet
- `dateCreated` : the date for which the ad at ebay was created
- `nrOfPictures` : number of pictures in the ad (unfortunately this field contains everywhere a 0 and is thus useless (bug in crawler!) )
- `postalCode`
- `lastSeenOnline` : when the crawler saw this ad last online

## 1. Setup
For analyzing the provided data, we'll use python with pandas and NumPy. Thus, we'll start by importing the libraries and reading the file.

In [2]:
import numpy as np
import pandas as pd

In [3]:
autos = pd.read_csv("autos.csv", encoding="Latin-1")

Now let's view the data.

In [4]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

In [5]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


As we can see, we have 20 columns with most of them being string columns. Looking at the types also shows the need for data cleaning here.

## 2. Cleaning the columns
Concerning the column-naming we can see that the data doesn't follow Python's conventions. E.g. it uses camelcase instead of snakecase.

In [6]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

Let's clean up these columns first. Firstly, we'll rename
- `yearOfRegistration` to `registration_year`
- `monthOfRegistration` to `registration_month`
- `notRepairedDamage` to `unrepaired_damage`
- `dateCreated` to `ad_created`
to make it more understandable. Secondly, we will change all camelcase names to snakecase.

In [7]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## 3. Initial Data Cleaning
Let's do some data exploration now. We are looking for text columns where all or almost all values are the same (they may be dropped due to no useful information) and for numeric data stored as text.

In [8]:
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-29 23:42:13,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


The columns `seller` and `offer_type` both contain only two values with only one different outlier. Basically all offers are private and an "Angebot".

`price` and `odometer` certainly are a numeric values stored as string.

`registration year` seems to be stored as number but is actually a date.

`nr_of_pictures` seems a little odd here.

In [9]:
autos["nr_of_pictures"].value_counts()

0    50000
Name: nr_of_pictures, dtype: int64

Well, as it seems, `nr_of_pictures` only has a value of 0 for every column. We can easily drop this one along with `seller` and `offer_type`

In [10]:
autos = autos.drop(["nr_of_pictures", "seller", "offer_type"], axis=1)

Now we can clean the `price` and `odometer` columns and store them as numeric values. 

In [11]:
new_prices = (autos["price"]
                         .str.replace("$","")
                         .str.replace(",","")
                         .astype(int)
             )
autos["price"] = new_prices
autos["price"].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

In [12]:
new_odometer = (autos["odometer"]
                         .str.replace("km","")
                         .str.replace(",","")
                         .astype(int)
             )
autos["odometer"] = new_odometer
autos["odometer"].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer, dtype: int64

We will also rename the odometer cell to `odometer_km`.

In [13]:
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)

## 4. Exploring odometer and price
After converting `odometer_km` and `price` to numeric values, let's explore the dataset for values that don't seem right.

In [17]:
print(autos["odometer_km"].value_counts())
autos["odometer_km"].describe()

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64


count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

The odometer values seem to be chosen from pre-set values. The steps are 5.000 to 10.000 kilometers. Moreover, most cars have high mileage – especially more than 150.000 km.

In [19]:
print(autos["price"].describe())
autos["price"].value_counts().head(10)

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
Name: price, dtype: int64

There seems to be a number of listings with a price of 0 $. In contrary to the odometer values, these values don't seem to be pre-set but free-to-choose though. 

In [21]:
print(autos["price"].value_counts().sort_index(ascending=True).head(10))
autos["price"].value_counts().sort_index(ascending=False).head(10)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
Name: price, dtype: int64


99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
Name: price, dtype: int64

Aside from about 1.500 0-dollar listings, there are also a number of listings with ridiculously high prices (e.g. 10 million dollars). In order to stop these values from interfering with our analysis, we will set an upper boundary of 1.000.000 dollars and remove the 0-dollar values.

In [22]:
autos = autos[autos["price"].between(1,1000000)]
autos["price"].describe()

count     48568.000000
mean       5950.340656
std       11963.134750
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      999999.000000
Name: price, dtype: float64

## 5. Exploring the date columns
Our dataset contains a number of columns with date information:
- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

Right now, the `date_crawled`, `last_seen`, and `ad_created` columns are all identified as string values. 

We'll explore each of these columns to better understand our data.

In [23]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [26]:
(autos["date_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index(ascending=True)
        )

2016-03-05    0.025325
2016-03-06    0.014042
2016-03-07    0.036011
2016-03-08    0.033294
2016-03-09    0.033088
2016-03-10    0.032182
2016-03-11    0.032573
2016-03-12    0.036917
2016-03-13    0.015669
2016-03-14    0.036547
2016-03-15    0.034282
2016-03-16    0.029608
2016-03-17    0.031646
2016-03-18    0.012910
2016-03-19    0.034776
2016-03-20    0.037885
2016-03-21    0.037391
2016-03-22    0.032985
2016-03-23    0.032223
2016-03-24    0.029340
2016-03-25    0.031605
2016-03-26    0.032202
2016-03-27    0.031090
2016-03-28    0.034858
2016-03-29    0.034117
2016-03-30    0.033685
2016-03-31    0.031832
2016-04-01    0.033685
2016-04-02    0.035476
2016-04-03    0.038606
2016-04-04    0.036485
2016-04-05    0.013095
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

As we can see, the listing seem to be scraped between March and April of 2016 for the period of a month.

In [27]:
(autos["date_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values(ascending=False)
        )

2016-04-03    0.038606
2016-03-20    0.037885
2016-03-21    0.037391
2016-03-12    0.036917
2016-03-14    0.036547
2016-04-04    0.036485
2016-03-07    0.036011
2016-04-02    0.035476
2016-03-28    0.034858
2016-03-19    0.034776
2016-03-15    0.034282
2016-03-29    0.034117
2016-04-01    0.033685
2016-03-30    0.033685
2016-03-08    0.033294
2016-03-09    0.033088
2016-03-22    0.032985
2016-03-11    0.032573
2016-03-23    0.032223
2016-03-26    0.032202
2016-03-10    0.032182
2016-03-31    0.031832
2016-03-17    0.031646
2016-03-25    0.031605
2016-03-27    0.031090
2016-03-16    0.029608
2016-03-24    0.029340
2016-03-05    0.025325
2016-03-13    0.015669
2016-03-06    0.014042
2016-04-05    0.013095
2016-03-18    0.012910
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

The value distribution seems to be rather uniform. It clearly can be observed that the scraping stopped on the 7th of April.

In [28]:
(autos["last_seen"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index(ascending=True)
        )

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005394
2016-03-08    0.007412
2016-03-09    0.009595
2016-03-10    0.010665
2016-03-11    0.012374
2016-03-12    0.023781
2016-03-13    0.008895
2016-03-14    0.012601
2016-03-15    0.015875
2016-03-16    0.016451
2016-03-17    0.028084
2016-03-18    0.007351
2016-03-19    0.015833
2016-03-20    0.020651
2016-03-21    0.020631
2016-03-22    0.021372
2016-03-23    0.018531
2016-03-24    0.019766
2016-03-25    0.019210
2016-03-26    0.016801
2016-03-27    0.015648
2016-03-28    0.020878
2016-03-29    0.022360
2016-03-30    0.024769
2016-03-31    0.023781
2016-04-01    0.022793
2016-04-02    0.024914
2016-04-03    0.025202
2016-04-04    0.024481
2016-04-05    0.124753
2016-04-06    0.221813
2016-04-07    0.131939
Name: last_seen, dtype: float64

In [29]:
(autos["last_seen"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values(ascending=False)
        )

2016-04-06    0.221813
2016-04-07    0.131939
2016-04-05    0.124753
2016-03-17    0.028084
2016-04-03    0.025202
2016-04-02    0.024914
2016-03-30    0.024769
2016-04-04    0.024481
2016-03-12    0.023781
2016-03-31    0.023781
2016-04-01    0.022793
2016-03-29    0.022360
2016-03-22    0.021372
2016-03-28    0.020878
2016-03-20    0.020651
2016-03-21    0.020631
2016-03-24    0.019766
2016-03-25    0.019210
2016-03-23    0.018531
2016-03-26    0.016801
2016-03-16    0.016451
2016-03-15    0.015875
2016-03-19    0.015833
2016-03-27    0.015648
2016-03-14    0.012601
2016-03-11    0.012374
2016-03-10    0.010665
2016-03-09    0.009595
2016-03-13    0.008895
2016-03-08    0.007412
2016-03-18    0.007351
2016-03-07    0.005394
2016-03-06    0.004324
2016-03-05    0.001071
Name: last_seen, dtype: float64

The `last_seen` column generally indicates when a listing was last seen. When it's not on the website anymore, it was probably sold in whatever way. Our dataset shows a large spike in these values during the last three days of scraping period.

While it could be that the sales were boosted this highly all of a sudden, it's more likely that the end of the scraping also determined the `last_seen` value and thus created these spikes.

In [30]:
(autos["ad_created"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index(ascending=True)
        )

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033149
2016-03-10    0.031893
2016-03-11    0.032902
2016-03-12    0.036753
2016-03-13    0.017007
2016-03-14    0.035188
2016-03-15    0.034014
2016-03-16    0.030123
2016-03-17    0.031296
2016-03-18    0.013589
2016-03-19    0.033685
2016-03-20    0.037947
2016-03-21 

The `ad_created` dates show much more variety than the other date column. While most of the listings seem to have been created during the scraping the-period, some of them go back months or even closely up to a year. 

In [31]:
autos[autos["ad_created"].str[:10] == "2015-06-11"]

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
22781,2016-03-23 01:48:59,Mercedes_Benz_C220_BT__7G_Tronic_AMG__Modellja...,47900,test,limousine,2014,automatik,0,c_klasse,20000,2,diesel,mercedes_benz,NaN,2015-06-11 00:00:00,46145,2016-04-06 09:47:02


In [32]:
autos["registration_year"].describe()

count    48568.000000
mean      2004.754612
std         88.641262
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

While most cars seem to be registered somewhere between 1999 and 2008, there are some odd values. The minimum value is `1000`, the highest `9999` which just doesn't make sense for year-values.

## 6. Cleaning the registration data
Because the dataset was collected in 2016, registration values above 2016 are not possible. So are early values. For the cleanup, we might use a valid registratioon interval of 1900 until 2016.

In [33]:
(~autos["registration_year"].between(1900,2016)).sum() / autos.shape[0]

0.038790973480480974

Given that values that fall outside these extremes make up approximately 3.9 % of the data, we can remove them.

In [35]:
autos = autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].describe()

count    46684.000000
mean      2002.910033
std          7.186122
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

Now, the values give us a much clearer picture. The oldest car was registered in 1910, the newest in 2016. Generally, most cars seem to be between 10 and 20 years old.

## 7. Exploring the brand column
To understand the car market on eBay better, we can analyze which brands are sold for which prices.

In [37]:
autos["brand"].value_counts(normalize=True)

volkswagen        0.211293
bmw               0.110038
opel              0.107574
mercedes_benz     0.096457
audi              0.086561
ford              0.069917
renault           0.047147
peugeot           0.029839
fiat              0.025640
seat              0.018272
skoda             0.016408
nissan            0.015273
mazda             0.015187
smart             0.014159
citroen           0.014009
toyota            0.012702
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008761
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006640
porsche           0.006126
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001499
trabant           0.001392
r

Funnily enough, German brands are hugely present in the dataset making up almost 50 % of the values. Especially Volkswagen is vastly popular.

For our pricing analysis, we want to take the most popular brands while also being as broad as possible. This, we'll cap our popular brands at 2 % listing-share.

In [40]:
brand_shares = autos["brand"].value_counts(normalize=True)
popular_brands = brand_shares[brand_shares > 0.02].index
popular_brands

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat'],
      dtype='object')

In [61]:
brand_mean_prices = {}

for brand in popular_brands:
    brand_cars = autos[autos["brand"] == brand]
    mean_price = brand_cars["price"].mean()
    brand_mean_prices[brand] = int(mean_price)
    
brand_mean_prices

{'audi': 9336,
 'bmw': 8332,
 'fiat': 2813,
 'ford': 4054,
 'mercedes_benz': 8628,
 'opel': 2975,
 'peugeot': 3094,
 'renault': 2474,
 'volkswagen': 5604}

From the evaluation of the top nine brands we can see:
- Audi, BMW and Mercedes Benz are generally quite expensive
- Volkswagen and Ford lie somewhere in the middle
- Fiat, Opel, Peugeot and Renault are the most affordable brands

## 8. Exploring mileage
Now that we know the average prices, we should also explore the mean mileages to see if there's any connection between those two.

In order to realize that, we will convert our dictionaries to new pandas series and a datraframe and visually compare these values. 

In [62]:
brand_mean_mileage = {}

for brand in popular_brands:
    brand_cars = autos[autos["brand"] == brand]
    mean_mileage = brand_cars["odometer_km"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)
    
brand_mean_mileage

{'audi': 129157,
 'bmw': 132572,
 'fiat': 117121,
 'ford': 124266,
 'mercedes_benz': 130788,
 'opel': 129310,
 'peugeot': 127153,
 'renault': 128071,
 'volkswagen': 128711}

In [63]:
brand_price_series = pd.Series(brand_mean_prices)
brand_info = pd.DataFrame(brand_price_series, columns=["mean_price"])
brand_info

,mean_price
audi,9336
bmw,8332
fiat,2813
ford,4054
mercedes_benz,8628
opel,2975
peugeot,3094
renault,2474
volkswagen,5604


In [64]:
brand_mileage_series = pd.Series(brand_mean_mileage)
brand_mileage_series

audi             129157
bmw              132572
fiat             117121
ford             124266
mercedes_benz    130788
opel             129310
peugeot          127153
renault          128071
volkswagen       128711
dtype: int64

In [65]:
brand_info["mean_mileage"] = brand_mileage_series
brand_info

,mean_price,mean_mileage
audi,9336,129157
bmw,8332,132572
fiat,2813,117121
ford,4054,124266
mercedes_benz,8628,130788
opel,2975,129310
peugeot,3094,127153
renault,2474,128071
volkswagen,5604,128711


Clearly, there's no real connection between the mileage and the average prices of different brands. They're approximately all the same. Thus, we can conclude that the brand does indeed largely determine the price on the German market for used cars.

## 9. Futher steps
Given I had time and reason to futher explore this dataset, I would:
- Find the most common brand/model combinations
- Split the `odometer_km` into groups and see if average prices follows any patterns based on the mileage.
- Figure out how much cheaper  cars with damage than their non-damaged counterparts are.